In [1]:
!pwd

C:\Users\srbhc\Documents\GoogleDrive\UCSD\Fall 2016\218\Project\SelfTutorGlass


In [1]:
import argparse
import base64
import os
import re
import sys

from googleapiclient import discovery
from googleapiclient import errors
from oauth2client.client import GoogleCredentials
import redis

DISCOVERY_URL = 'https://{api}.googleapis.com/$discovery/rest?version={apiVersion}'  # noqa
os.environ['GOOGLE_APPLICATION_CREDENTIALS']='saurabh_google_api_credential.json'

In [2]:
class VisionApi:
    """Construct and use the Google Vision API service."""

    def __init__(self, api_discovery_file='vision_api.json'):
        self.credentials = GoogleCredentials.get_application_default()
        self.service = discovery.build(
            'vision', 'v1', credentials=self.credentials,
            discoveryServiceUrl=DISCOVERY_URL)

    def detect_text(self, input_filenames, num_retries=3, max_results=6):
        """Uses the Vision API to detect text in the given file.
        """
        images = {}
        for filename in input_filenames:
            with open(filename, 'rb') as image_file:
                images[filename] = image_file.read()

        batch_request = []
        for filename in images:
            batch_request.append({
                'image': {
                    'content': base64.b64encode(
                            images[filename]).decode('UTF-8')
                },
                'features': [{
                    'type': 'TEXT_DETECTION',
                    'maxResults': max_results,
                }]
            })
        request = self.service.images().annotate(
            body={'requests': batch_request})

        try:
            responses = request.execute(num_retries=num_retries)
            if 'responses' not in responses:
                return {}
            text_response = {}
            for filename, response in zip(images, responses['responses']):
                if 'error' in response:
                    print("API Error for %s: %s" % (
                            filename,
                            response['error']['message']
                            if 'message' in response['error']
                            else ''))
                    continue
                if 'textAnnotations' in response:
                    text_response[filename] = response['textAnnotations']
                else:
                    text_response[filename] = []
            return text_response
        except errors.HttpError as e:
            print("Http Error for %s: %s" % (filename, e))
        except KeyError as e2:
            print("Key error: %s" % e2)

In [3]:
# Create a client object for the Vision API
vision = VisionApi()

fileslist = ['demo-image.jpg', 'language_images/images.png', 'language_images/english_alphabets.jpg']

texts = vision.detect_text(fileslist)
for k,v in texts.items():
    print(k, v)

language_images/english_alphabets.jpg [{'description': 'ABCDEFG\nHIJKLN\nOPQRSTU\nVWXYZ\nM\nDKR\n', 'locale': 'da', 'boundingPoly': {'vertices': [{'y': 18, 'x': 19}, {'y': 18, 'x': 636}, {'y': 354, 'x': 636}, {'y': 354, 'x': 19}]}}, {'description': 'ABCDEFG', 'boundingPoly': {'vertices': [{'y': 19, 'x': 19}, {'y': 19, 'x': 633}, {'y': 51, 'x': 633}, {'y': 51, 'x': 19}]}}, {'description': 'HIJKLN', 'boundingPoly': {'vertices': [{'y': 119, 'x': 23}, {'y': 119, 'x': 633}, {'y': 151, 'x': 633}, {'y': 151, 'x': 23}]}}, {'description': 'OPQRSTU', 'boundingPoly': {'vertices': [{'y': 223, 'x': 22}, {'y': 223, 'x': 636}, {'y': 255, 'x': 636}, {'y': 255, 'x': 22}]}}, {'description': 'VWXYZ', 'boundingPoly': {'vertices': [{'y': 323, 'x': 19}, {'y': 323, 'x': 443}, {'y': 354, 'x': 443}, {'y': 354, 'x': 19}]}}, {'description': 'M', 'boundingPoly': {'vertices': [{'y': 121, 'x': 501}, {'y': 118, 'x': 542}, {'y': 153, 'x': 544}, {'y': 156, 'x': 503}]}}, {'description': 'DKR', 'boundingPoly': {'vertice